In [4]:
import pandas as pd

env_data = pd.read_csv('../.env')
ENV = {}
for row in env_data[env_data.columns[0]]:
    #key, value = 
    row_data = row.split('=')
    key = row_data[0]
    value = '='.join(row_data[1:])
    value = value.replace("'", "").replace('"', '')
    ENV.update({key: value})

ENV

{'INSTANCE_UNIX_SOCKET': '/cloudsql/animated-splice-364318:us-central1:artica-main-storage',
 'DB_USER': 'root',
 'DB_PASS': 'Ym#[>~=Bf:1o.ka)',
 'INSTANCE_CONNECTION_NAME': 'animated-splice-364318:us-central1:artica-main-storage'}

In [5]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql

# initialize Connector object
connector = Connector()

SELECTED_DB_NAME = "COMMON_DATA"
# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        ENV['INSTANCE_CONNECTION_NAME'],
        "pymysql",
        user=ENV['DB_USER'],
        password=ENV['DB_PASS'],
        db=SELECTED_DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql:///COMMON_DATA.db",
    creator=getconn,
    echo = True
)

In [6]:
# reading
show_dbs_statement = sqlalchemy.text('SHOW DATABASES;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

d:\repos\python\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


2023-03-11 15:39:54,823 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-11 15:39:54,825 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 15:39:55,937 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-11 15:39:55,940 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 15:39:56,222 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-11 15:39:56,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-11 15:39:56,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-11 15:39:56,768 INFO sqlalchemy.engine.Engine SHOW DATABASES;
2023-03-11 15:39:56,771 INFO sqlalchemy.engine.Engine [generated in 0.00593s] {}
2023-03-11 15:39:57,098 INFO sqlalchemy.engine.Engine ROLLBACK


,Database
0,1A
1,COMMON_DATA
2,ORDER1
3,information_schema
4,mysql
5,performance_schema
6,sys


In [11]:
# reading
show_dbs_statement = sqlalchemy.text('SELECT * FROM orders;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-11 15:49:29,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-11 15:49:29,634 INFO sqlalchemy.engine.Engine SELECT * FROM orders;
2023-03-11 15:49:29,636 INFO sqlalchemy.engine.Engine [generated in 0.00410s] {}
2023-03-11 15:49:29,918 INFO sqlalchemy.engine.Engine ROLLBACK


,orderId,site,unitCode,createDate,timeZone
0,1,CENTURY ENKA BHOSARI,CPRG160/17.5/1001,2023-03-10 16:26:29,Asia/Kolkata


In [134]:
# drop table orders

# insert_stmt = sqlalchemy.text(
#     "DROP TABLE COMMON_DATA.orders;",
# )

# with pool.connect() as db_conn:
#     # insert into database
#     db_conn.execute(insert_stmt)
#     db_conn.commit()

2023-03-10 16:23:01,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 16:23:01,281 INFO sqlalchemy.engine.Engine DROP TABLE COMMON_DATA.orders;
2023-03-10 16:23:01,281 INFO sqlalchemy.engine.Engine [generated in 0.00110s] {}
2023-03-10 16:23:01,623 INFO sqlalchemy.engine.Engine COMMIT


In [136]:
import pytz

max([len(tz) for tz in pytz.all_timezones])

32

In [137]:
# create table orders
meta = sqlalchemy.MetaData()

orders = sqlalchemy.Table(
    'orders', meta, 
    sqlalchemy.Column('orderId', sqlalchemy.Integer, primary_key = True), 
    sqlalchemy.Column('site', sqlalchemy.String(30)), 
    sqlalchemy.Column('unitCode', sqlalchemy.String(30)),
    sqlalchemy.Column('createDate', sqlalchemy.DATETIME),
    sqlalchemy.Column('timeZone', sqlalchemy.TEXT(50)),
)

meta.create_all(pool)

2023-03-10 16:25:35,021 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 16:25:35,023 INFO sqlalchemy.engine.Engine DESCRIBE `COMMON_DATA`.`orders`
2023-03-10 16:25:35,023 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-10 16:25:35,372 INFO sqlalchemy.engine.Engine 
CREATE TABLE orders (
	`orderId` INTEGER NOT NULL AUTO_INCREMENT, 
	site VARCHAR(30), 
	`unitCode` VARCHAR(30), 
	`createDate` DATETIME, 
	`timeZone` TEXT(50), 
	PRIMARY KEY (`orderId`)
)


2023-03-10 16:25:35,373 INFO sqlalchemy.engine.Engine [no key 0.00117s] {}
2023-03-10 16:25:35,719 INFO sqlalchemy.engine.Engine COMMIT


In [143]:
# insert first order details in order table

from datetime import datetime

data = {
    'orderId': 1,
    'site': 'CENTURY ENKA BHOSARI',
    'unitCode': 'CPRG160/17.5/1001',
    'createDate': datetime.now(),
    'timeZone': 'Asia/Kolkata'
}

insert_stmt = sqlalchemy.text(
    "INSERT INTO orders (orderId, site, unitCode, createDate, timeZone) VALUES (:orderId, :site, :unitCode, :createDate, :timeZone) ON DUPLICATE KEY UPDATE site=:site, unitCode=:unitCode, timeZone=:timeZone;",
)

with pool.connect() as db_conn:
    # insert into database
    db_conn.execute(insert_stmt, parameters=data)
    db_conn.commit()

2023-03-10 16:28:15,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 16:28:15,833 INFO sqlalchemy.engine.Engine INSERT INTO orders (orderId, site, unitCode, createDate, timeZone) VALUES (%(orderId)s, %(site)s, %(unitCode)s, %(createDate)s, %(timeZone)s) ON DUPLICATE KEY UPDATE site=%(site)s, unitCode=%(unitCode)s, timeZone=%(timeZone)s;
2023-03-10 16:28:15,833 INFO sqlalchemy.engine.Engine [generated in 0.00185s] {'orderId': 1, 'site': 'CENTURY ENKA BHOSARI', 'unitCode': 'CPRG160/17.5/1001', 'createDate': datetime.datetime(2023, 3, 10, 16, 28, 15, 831400), 'timeZone': 'Asia/Kolkata'}
2023-03-10 16:28:16,209 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
# DEFINE UNIT TYPES

unitsDf = pd.read_csv('unitConversionReference.csv')

units_records = unitsDf.to_dict(orient='records')

In [40]:
# remove table units

# insert_stmt = sqlalchemy.text(
#     "DROP TABLE units;",
# )

# with pool.connect() as db_conn:
#     # insert into database
#     db_conn.execute(insert_stmt)
#     db_conn.commit()

2023-03-10 11:25:33,489 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 11:25:33,489 INFO sqlalchemy.engine.Engine DROP TABLE units;
2023-03-10 11:25:33,490 INFO sqlalchemy.engine.Engine [cached since 1375s ago] {}
2023-03-10 11:25:33,798 INFO sqlalchemy.engine.Engine COMMIT


In [41]:
# CREATE TABLE units

meta = sqlalchemy.MetaData()

units = sqlalchemy.Table(
   'units', meta, 
   sqlalchemy.Column('unitId', sqlalchemy.Integer, primary_key = True), 
   sqlalchemy.Column('factor', sqlalchemy.REAL), 
   sqlalchemy.Column('bias', sqlalchemy.REAL),
   sqlalchemy.Column('sourceUnit', sqlalchemy.TEXT(127)),
   sqlalchemy.Column('comment', sqlalchemy.TEXT(255))  
)

meta.create_all(pool)

2023-03-10 11:25:36,677 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 11:25:36,678 INFO sqlalchemy.engine.Engine DESCRIBE `COMMON_DATA`.`units`
2023-03-10 11:25:36,679 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-10 11:25:37,027 INFO sqlalchemy.engine.Engine 
CREATE TABLE units (
	`unitId` INTEGER NOT NULL AUTO_INCREMENT, 
	factor REAL, 
	bias REAL, 
	`sourceUnit` TEXT(127), 
	comment TEXT(255), 
	PRIMARY KEY (`unitId`)
)


2023-03-10 11:25:37,029 INFO sqlalchemy.engine.Engine [no key 0.00140s] {}
2023-03-10 11:25:37,517 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
# reading
show_dbs_statement = sqlalchemy.text('SELECT orderId, timeZone from orders;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    timeZonesByOrderId = {row[0]: row[1] for row in data}

timeZonesByOrderId

2023-03-11 15:41:04,903 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-11 15:41:04,905 INFO sqlalchemy.engine.Engine SELECT orderId, timeZone from orders;
2023-03-11 15:41:04,907 INFO sqlalchemy.engine.Engine [generated in 0.00364s] {}
2023-03-11 15:41:05,175 INFO sqlalchemy.engine.Engine ROLLBACK


{1: 'Asia/Kolkata'}

In [43]:
# upload units data into units table

with pool.connect() as conn:
    result = conn.execute(
        sqlalchemy.insert(units).values(units_records),
    )
    conn.commit()

2023-03-10 11:25:48,282 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 11:25:48,283 INFO sqlalchemy.engine.Engine INSERT INTO units (`unitId`, factor, bias, `sourceUnit`, comment) VALUES (%(unitId)s, %(factor)s, %(bias)s, %(sourceUnit)s, %(comment)s)
2023-03-10 11:25:48,284 INFO sqlalchemy.engine.Engine [generated in 0.00189s] [{'unitId': 1, 'factor': 1.0, 'bias': 0.0, 'sourceUnit': 'No conversion', 'comment': 'default - No conversion'}, {'unitId': 2, 'factor': 98066.5, 'bias': 0.0, 'sourceUnit': 'kg/cm2', 'comment': 'kg/cm2 to Pasacal'}, {'unitId': 3, 'factor': 0.000278, 'bias': 0.0, 'sourceUnit': 'kg/h', 'comment': 'mass flow from kg/h to kg/s'}, {'unitId': 4, 'factor': 6894.76, 'bias': 0.0, 'sourceUnit': 'psi', 'comment': 'psi to Pasacal'}, {'unitId': 5, 'factor': 98066.5, 'bias': 101325.0, 'sourceUnit': 'kg/cm2-guage', 'comment': 'kg/cm2 (guage pressure) to Pasacal (absolute pressure)'}, {'unitId': 6, 'factor': -1.0, 'bias': 100.0, 'sourceUnit': '% opening', 'comment': '

In [10]:
# checking if units is filled by reading it

show_dbs_statement = sqlalchemy.text('SELECT * FROM units;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)


2023-03-11 15:42:10,417 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-11 15:42:10,419 INFO sqlalchemy.engine.Engine SELECT * FROM units;
2023-03-11 15:42:10,421 INFO sqlalchemy.engine.Engine [generated in 0.00427s] {}
2023-03-11 15:42:10,691 INFO sqlalchemy.engine.Engine ROLLBACK


,unitId,factor,bias,sourceUnit,comment
0,1,1.000000,0.00,No conversion,default - No conversion
1,2,98066.500000,0.00,kg/cm2,kg/cm2 to Pasacal
2,3,0.000278,0.00,kg/h,mass flow from kg/h to kg/s
3,4,6894.760000,0.00,psi,psi to Pasacal
4,5,98066.500000,101325.00,kg/cm2-guage,kg/cm2 (guage pressure) to Pasacal (absolute p...
5,6,-1.000000,100.00,% opening,%opening to % closing (y = 100-x)
6,7,1.000000,273.15,degC,degC to Kelvin
7,8,60.000000,0.00,min,Minutes to seconds
8,9,3600.000000,0.00,hr,Hours to seconds
9,10,4184.000000,0.00,kcal/kg,kcal/kg to J/kg


In [127]:
unitsReference = {row[0]: {'factor': row[1], 'bias': row[2]} for row in data}

unitsReference

{1: {'factor': 1.0, 'bias': 0.0},
 2: {'factor': 98066.5, 'bias': 0.0},
 3: {'factor': 0.000278, 'bias': 0.0},
 4: {'factor': 6894.76, 'bias': 0.0},
 5: {'factor': 98066.5, 'bias': 101325.0},
 6: {'factor': -1.0, 'bias': 100.0},
 7: {'factor': 1.0, 'bias': 273.15},
 8: {'factor': 60.0, 'bias': 0.0},
 9: {'factor': 3600.0, 'bias': 0.0},
 10: {'factor': 4184.0, 'bias': 0.0},
 11: {'factor': 1000.0, 'bias': 0.0}}

In [ ]:
# create database for new order

insert_stmt = sqlalchemy.text(
    "CREATE DATABASE ORDER1;",
)

with pool.connect() as db_conn:
    # insert into database
    db_conn.execute(insert_stmt)
    db_conn.commit()

2023-03-10 10:27:31,521 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 10:27:31,523 INFO sqlalchemy.engine.Engine CREATE DATABASE ORDER1;
2023-03-10 10:27:31,523 INFO sqlalchemy.engine.Engine [generated in 0.00198s] {}
2023-03-10 10:27:31,831 INFO sqlalchemy.engine.Engine COMMIT


In [36]:
# load meta data from xlsx file

metaDf = pd.read_excel('centuryEnkaMetaData-standard.xlsx')
metaRecords = metaDf.to_dict(orient='records')

In [75]:
META_TABLE_COLUMNS = ['standardTag', 'description', 'dataTagId', 'unitId']
metaTableRecords = [{key: record[key] for key in META_TABLE_COLUMNS} for record in metaRecords]

pd.DataFrame(metaTableRecords)

,standardTag,description,dataTagId,unitId
0,PROSS1_INN_STEAM_PRSS1,Process Inlet Steam Pressure,PT2_7_SCALE_PV,2
1,FURNC1_INN_FUELS_TOTZ1,Furnace Inlet Totalized Fuel,TOTALISE_FUEL_FLOW,11
2,PROSS1_INN_STEAM_TOTZ1,Process Inlet Totalized Steam,TOTAL_STEAM,11
3,FWTNK1_OUT_WATER_TEMP1,Process Outlet/Make up Water Temperature,TE1_5_SCALE_PV,7
4,PROSS1_INN_STEAM_TEMP1,Process Inlet Exit Steam Temperature,TE2_8_SCALE_PV,7
5,CTRLX1_MID_XXXXX_LOAD1,System Load,BOILER_LOAD,1
6,BOILR1_INN_WATER_FLOW1,Feed water to boiler -- mass flow rate,FEED_WATER_FLOW,3
7,BOILR1_INN_WATER_TEMP1,Feed water to boiler -- temperature,TE1_5_SCALE_PV,7
8,BOILR1_MID_DRUMX_PRSS1,Boiler tank pressure,PT2_7_SCALE_PV,2
9,BOILR1_OUT_STEAM_FLOW1,Boiler drum steam-out mass flow rate,FT2_14A_SCALE_PV,3


In [57]:
FIXED_TABLE_COLUMNS = ['standardTag', 'description','value', 'unitId']

constantsDf = pd.read_excel('centuryEnkaConstantsData.xlsx')
constantsRecords = constantsDf.to_dict(orient='records')

fixedTableRecords = [{key: record[key] for key in FIXED_TABLE_COLUMNS} for record in constantsRecords]

pd.DataFrame(fixedTableRecords)

,standardTag,description,value,unitId
0,DREFF1_MID_RLAVG_TIME1,Direct Efficiency Averaging Time Window,60.000000,8
1,FURNC1_INN_FUELS_HGCV1,Gross calorific value of fuel in use,3898.000000,10
2,FURNC1_INN_FUELS_HNCV1,Net calorific value of fuel in use,3620.000000,10
3,SCOMB1_MID_FUELS_COFW1,Fuel profile constant term,1.002000,1
4,SCOMB1_MID_FUELS_COFW2,Fuel profile coefficient of exponential term,0.981000,1
5,SCOMB1_MID_FUELS_COFW3,Fuel profile coefficient of timestep,0.001210,1
6,HEATX1_MID_XXXXX_COFW4,Heat Transfer Efficiency,0.833000,1
7,HEATX1_MID_XXXXX_COFW5,Coefficiency W5 - Load vs X_F,0.416000,1
8,HEATX1_MID_XXXXX_COFW6,Coefficiency W6 - Load vs X_F,0.105000,1
9,HEATX1_MID_XXXXX_COFW7,Coefficiency W7 - Load vs X_F,0.053000,1


In [129]:
orderId = 1

# create connection pool for new order's database
SELECTED_DB_NAME = 'ORDER1'

order_data_pool = sqlalchemy.create_engine(
    f"mysql+pymysql:///{SELECTED_DB_NAME}.db",
    creator=getconn,
    echo = True
)

In [97]:
# reading
show_dbs_statement = sqlalchemy.text('SHOW TABLES;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 13:10:37,504 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 13:10:37,505 INFO sqlalchemy.engine.Engine SHOW TABLES;
2023-03-10 13:10:37,506 INFO sqlalchemy.engine.Engine [cached since 1824s ago] {}
2023-03-10 13:10:37,814 INFO sqlalchemy.engine.Engine ROLLBACK


,Tables_in_ORDER1
0,FIXED_DATA
1,META_DATA
2,TEMP_DATA


In [80]:
{
    'length of standardTag' : len(metaTableRecords[0]['standardTag']),
    'metaColumns': META_TABLE_COLUMNS,
    'fixedColumns': FIXED_TABLE_COLUMNS
}

{'length of standardTag': 22,
 'metaColumns': ['standardTag', 'description', 'dataTagId', 'unitId'],
 'fixedColumns': ['standardTag', 'description', 'value', 'unitId']}

In [111]:
# create tables
meta = sqlalchemy.MetaData()

metaData = sqlalchemy.Table(
	'META_DATA', meta,  
	sqlalchemy.Column('standardTag', sqlalchemy.String(30), primary_key = True), 
	sqlalchemy.Column('description', sqlalchemy.TEXT(255)),
	sqlalchemy.Column('dataTagId', sqlalchemy.TEXT(127)),
	sqlalchemy.Column('unitId', sqlalchemy.INTEGER) 
)

fixedData = sqlalchemy.Table(
	'FIXED_DATA', meta, 
	sqlalchemy.Column('standardTag', sqlalchemy.String(30), primary_key = True), 
	sqlalchemy.Column('description', sqlalchemy.TEXT(255)),
	sqlalchemy.Column('value', sqlalchemy.REAL),
	sqlalchemy.Column('unitId', sqlalchemy.INTEGER) 
)

meta.create_all(order_data_pool)

2023-03-10 14:06:20,815 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:06:20,816 INFO sqlalchemy.engine.Engine DESCRIBE `ORDER1`.`META_DATA`
2023-03-10 14:06:20,816 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-10 14:06:21,130 INFO sqlalchemy.engine.Engine DESCRIBE `ORDER1`.`FIXED_DATA`
2023-03-10 14:06:21,131 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-10 14:06:21,439 INFO sqlalchemy.engine.Engine 
CREATE TABLE `META_DATA` (
	`standardTag` VARCHAR(30) NOT NULL, 
	description TEXT(255), 
	`dataTagId` TEXT(127), 
	`unitId` INTEGER, 
	PRIMARY KEY (`standardTag`)
)


2023-03-10 14:06:21,440 INFO sqlalchemy.engine.Engine [no key 0.00159s] {}
2023-03-10 14:06:21,810 INFO sqlalchemy.engine.Engine 
CREATE TABLE `FIXED_DATA` (
	`standardTag` VARCHAR(30) NOT NULL, 
	description TEXT(255), 
	value REAL, 
	`unitId` INTEGER, 
	PRIMARY KEY (`standardTag`)
)


2023-03-10 14:06:21,811 INFO sqlalchemy.engine.Engine [no key 0.00089s] {}
2023-03-10 14:06:22,122 INFO sqlalchemy.

In [112]:
# reading
show_dbs_statement = sqlalchemy.text('DESC META_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 14:06:28,259 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:06:28,260 INFO sqlalchemy.engine.Engine DESC META_DATA;
2023-03-10 14:06:28,260 INFO sqlalchemy.engine.Engine [generated in 0.00112s] {}
2023-03-10 14:06:28,610 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,standardTag,varchar(30),NO,PRI,None,
1,description,text,YES,,None,
2,dataTagId,text,YES,,None,
3,unitId,int,YES,,None,


In [113]:
# reading
show_dbs_statement = sqlalchemy.text('DESC FIXED_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 14:06:42,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:06:42,462 INFO sqlalchemy.engine.Engine DESC FIXED_DATA;
2023-03-10 14:06:42,464 INFO sqlalchemy.engine.Engine [generated in 0.00334s] {}
2023-03-10 14:06:42,822 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,standardTag,varchar(30),NO,PRI,None,
1,description,text,YES,,None,
2,value,double,YES,,None,
3,unitId,int,YES,,None,


In [114]:
# uploading meta-data

with order_data_pool.connect() as conn:
    result = conn.execute(
        sqlalchemy.insert(metaData),
        metaTableRecords
    )
    conn.commit()

# reading
show_dbs_statement = sqlalchemy.text('SELECT * FROM META_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 14:06:51,319 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:06:51,320 INFO sqlalchemy.engine.Engine INSERT INTO `META_DATA` (`standardTag`, description, `dataTagId`, `unitId`) VALUES (%(standardTag)s, %(description)s, %(dataTagId)s, %(unitId)s)
2023-03-10 14:06:51,320 INFO sqlalchemy.engine.Engine [generated in 0.00186s] [{'standardTag': 'PROSS1_INN_STEAM_PRSS1', 'description': 'Process Inlet Steam Pressure', 'dataTagId': 'PT2_7_SCALE_PV', 'unitId': 2}, {'standardTag': 'FURNC1_INN_FUELS_TOTZ1', 'description': 'Furnace Inlet Totalized Fuel', 'dataTagId': 'TOTALISE_FUEL_FLOW', 'unitId': 11}, {'standardTag': 'PROSS1_INN_STEAM_TOTZ1', 'description': 'Process Inlet Totalized Steam', 'dataTagId': 'TOTAL_STEAM', 'unitId': 11}, {'standardTag': 'FWTNK1_OUT_WATER_TEMP1', 'description': 'Process Outlet/Make up Water Temperature', 'dataTagId': 'TE1_5_SCALE_PV', 'unitId': 7}, {'standardTag': 'PROSS1_INN_STEAM_TEMP1', 'description': 'Process Inlet Exit Steam Temperature', 'd

,standardTag,description,dataTagId,unitId
0,BOILR1_INN_WATER_FLOW1,Feed water to boiler -- mass flow rate,FEED_WATER_FLOW,3
1,BOILR1_INN_WATER_TEMP1,Feed water to boiler -- temperature,TE1_5_SCALE_PV,7
2,BOILR1_MID_DRUMX_LEVL1,Boiler drum level transmitter output,LT2_4_SCALE_PV,1
3,BOILR1_MID_DRUMX_PRSS1,Boiler tank pressure,PT2_7_SCALE_PV,2
4,BOILR1_OUT_STEAM_FLOW1,Boiler drum steam-out mass flow rate,FT2_14A_SCALE_PV,3
5,CTRLX1_MID_XXXXX_LOAD1,System Load,BOILER_LOAD,1
6,FURNC1_INN_FUELS_TOTZ1,Furnace Inlet Totalized Fuel,TOTALISE_FUEL_FLOW,11
7,FWTNK1_OUT_WATER_TEMP1,Process Outlet/Make up Water Temperature,TE1_5_SCALE_PV,7
8,PROSS1_INN_STEAM_PRSS1,Process Inlet Steam Pressure,PT2_7_SCALE_PV,2
9,PROSS1_INN_STEAM_TEMP1,Process Inlet Exit Steam Temperature,TE2_8_SCALE_PV,7


In [133]:
# reading
show_dbs_statement = sqlalchemy.text('SELECT standardTag, dataTagId, unitId FROM META_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    data = result.fetchall()
    # resultDict = dict(zip(keys, zip(*data)))

metaDataDict = {metarow[0]: {'dataTagId:':metarow[1], 'unitId': metarow[2]} for metarow in data}
metaDataDict

2023-03-10 16:10:32,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 16:10:32,024 INFO sqlalchemy.engine.Engine SELECT standardTag, dataTagId, unitId FROM META_DATA;
2023-03-10 16:10:32,025 INFO sqlalchemy.engine.Engine [cached since 177.3s ago] {}
2023-03-10 16:10:32,384 INFO sqlalchemy.engine.Engine ROLLBACK


{'BOILR1_INN_WATER_FLOW1': {'dataTagId:': 'FEED_WATER_FLOW', 'unitId': 3},
 'BOILR1_INN_WATER_TEMP1': {'dataTagId:': 'TE1_5_SCALE_PV', 'unitId': 7},
 'BOILR1_MID_DRUMX_LEVL1': {'dataTagId:': 'LT2_4_SCALE_PV', 'unitId': 1},
 'BOILR1_MID_DRUMX_PRSS1': {'dataTagId:': 'PT2_7_SCALE_PV', 'unitId': 2},
 'BOILR1_OUT_STEAM_FLOW1': {'dataTagId:': 'FT2_14A_SCALE_PV', 'unitId': 3},
 'CTRLX1_MID_XXXXX_LOAD1': {'dataTagId:': 'BOILER_LOAD', 'unitId': 1},
 'FURNC1_INN_FUELS_TOTZ1': {'dataTagId:': 'TOTALISE_FUEL_FLOW', 'unitId': 11},
 'FWTNK1_OUT_WATER_TEMP1': {'dataTagId:': 'TE1_5_SCALE_PV', 'unitId': 7},
 'PROSS1_INN_STEAM_PRSS1': {'dataTagId:': 'PT2_7_SCALE_PV', 'unitId': 2},
 'PROSS1_INN_STEAM_TEMP1': {'dataTagId:': 'TE2_8_SCALE_PV', 'unitId': 7},
 'PROSS1_INN_STEAM_TOTZ1': {'dataTagId:': 'TOTAL_STEAM', 'unitId': 11}}

In [115]:
# uploading fixed-data

with order_data_pool.connect() as conn:
    result = conn.execute(
        sqlalchemy.insert(fixedData),
        fixedTableRecords
    )
    conn.commit()

# reading
show_dbs_statement = sqlalchemy.text('SELECT * FROM FIXED_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 14:07:03,953 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:07:03,954 INFO sqlalchemy.engine.Engine INSERT INTO `FIXED_DATA` (`standardTag`, description, value, `unitId`) VALUES (%(standardTag)s, %(description)s, %(value)s, %(unitId)s)
2023-03-10 14:07:03,955 INFO sqlalchemy.engine.Engine [generated in 0.00197s] [{'standardTag': 'DREFF1_MID_RLAVG_TIME1', 'description': 'Direct Efficiency Averaging Time Window', 'value': 60.0, 'unitId': 8}, {'standardTag': 'FURNC1_INN_FUELS_HGCV1', 'description': 'Gross calorific value of fuel in use', 'value': 3898.0, 'unitId': 10}, {'standardTag': 'FURNC1_INN_FUELS_HNCV1', 'description': 'Net calorific value of fuel in use', 'value': 3620.0, 'unitId': 10}, {'standardTag': 'SCOMB1_MID_FUELS_COFW1', 'description': 'Fuel profile constant term', 'value': 1.002, 'unitId': 1}, {'standardTag': 'SCOMB1_MID_FUELS_COFW2', 'description': 'Fuel profile coefficient of exponential term', 'value': 0.981, 'unitId': 1}, {'standardTag': 'SCOMB1

,standardTag,description,value,unitId
0,DREFF1_MID_RLAVG_TIME1,Direct Efficiency Averaging Time Window,60.000000,8
1,FURNC1_INN_FUELS_HGCV1,Gross calorific value of fuel in use,3898.000000,10
2,FURNC1_INN_FUELS_HNCV1,Net calorific value of fuel in use,3620.000000,10
3,HEATX1_MID_XXXXX_COFW4,Heat Transfer Efficiency,0.833000,1
4,HEATX1_MID_XXXXX_COFW5,Coefficiency W5 - Load vs X_F,0.416000,1
5,HEATX1_MID_XXXXX_COFW6,Coefficiency W6 - Load vs X_F,0.105000,1
6,HEATX1_MID_XXXXX_COFW7,Coefficiency W7 - Load vs X_F,0.053000,1
7,HYDRO1_MID_DNCMR_AREA1,Area of downcomer cross section,0.032283,1
8,HYDRO1_MID_DNCMR_CFRN1,dimensionless friction coefficienct Eq 16@Astr...,2333.807750,1
9,HYDRO1_MID_DNCMR_VOLM1,Volume of downcomer,0.326800,1


In [103]:
# create temp-data
meta = sqlalchemy.MetaData()

tempDataColumnsStr = [record['standardTag'] for record in metaTableRecords]
tempDataColumns = [sqlalchemy.Column(standardTag, sqlalchemy.REAL) for standardTag in tempDataColumnsStr]

metaData = sqlalchemy.Table(
	'TEMP_DATA', meta, 
    sqlalchemy.Column('t', sqlalchemy.DATETIME, primary_key=True),
	*tempDataColumns
)

meta.create_all(order_data_pool)

2023-03-10 13:15:59,604 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 13:15:59,605 INFO sqlalchemy.engine.Engine DESCRIBE `ORDER1`.`TEMP_DATA`
2023-03-10 13:15:59,605 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-10 13:15:59,966 INFO sqlalchemy.engine.Engine 
CREATE TABLE `TEMP_DATA` (
	t DATETIME NOT NULL, 
	`PROSS1_INN_STEAM_PRSS1` REAL, 
	`FURNC1_INN_FUELS_TOTZ1` REAL, 
	`PROSS1_INN_STEAM_TOTZ1` REAL, 
	`FWTNK1_OUT_WATER_TEMP1` REAL, 
	`PROSS1_INN_STEAM_TEMP1` REAL, 
	`CTRLX1_MID_XXXXX_LOAD1` REAL, 
	`BOILR1_INN_WATER_FLOW1` REAL, 
	`BOILR1_INN_WATER_TEMP1` REAL, 
	`BOILR1_MID_DRUMX_PRSS1` REAL, 
	`BOILR1_OUT_STEAM_FLOW1` REAL, 
	`BOILR1_MID_DRUMX_LEVL1` REAL, 
	PRIMARY KEY (t)
)


2023-03-10 13:15:59,967 INFO sqlalchemy.engine.Engine [no key 0.00151s] {}
2023-03-10 13:16:00,383 INFO sqlalchemy.engine.Engine COMMIT


In [104]:
# reading
show_dbs_statement = sqlalchemy.text('DESC TEMP_DATA;')

with order_data_pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-10 13:16:03,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 13:16:03,165 INFO sqlalchemy.engine.Engine DESC TEMP_DATA;
2023-03-10 13:16:03,166 INFO sqlalchemy.engine.Engine [cached since 310.6s ago] {}
2023-03-10 13:16:03,551 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,t,datetime,NO,PRI,None,
1,PROSS1_INN_STEAM_PRSS1,double,YES,,None,
2,FURNC1_INN_FUELS_TOTZ1,double,YES,,None,
3,PROSS1_INN_STEAM_TOTZ1,double,YES,,None,
4,FWTNK1_OUT_WATER_TEMP1,double,YES,,None,
5,PROSS1_INN_STEAM_TEMP1,double,YES,,None,
6,CTRLX1_MID_XXXXX_LOAD1,double,YES,,None,
7,BOILR1_INN_WATER_FLOW1,double,YES,,None,
8,BOILR1_INN_WATER_TEMP1,double,YES,,None,
9,BOILR1_MID_DRUMX_PRSS1,double,YES,,None,


In [110]:
# drop table temp_data

insert_stmt = sqlalchemy.text(
    "DROP TABLE FIXED_DATA;",
)

with order_data_pool.connect() as db_conn:
    # insert into database
    db_conn.execute(insert_stmt)
    db_conn.commit()

2023-03-10 14:05:38,894 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-10 14:05:38,895 INFO sqlalchemy.engine.Engine DROP TABLE FIXED_DATA;
2023-03-10 14:05:38,896 INFO sqlalchemy.engine.Engine [generated in 0.00237s] {}
2023-03-10 14:05:39,209 INFO sqlalchemy.engine.Engine COMMIT
